In [4]:
import pandas as pd
import requests

In [22]:
cc_metadata = pd.read_csv("../data/cc_metadata.csv", index_col=0)
cc_metadata.head(5)

,title,author,year,word_count,not_before,not_after,is_earlymodern
10,De facto Ungarie magne1236\n,Riccardus OFM,fl.1236,3340,1236.0,1236.0,False
100,De trinitate,Novatianus,fl. 260,44792,260.0,260.0,False
10000,De quadratura circuli,Franco Leodiensis,NaN,1648,NaN,NaN,False
10001,Epistola ad Berengarium,Frollandus Sylvanectensis,NaN,518,NaN,NaN,False
10002,Diplomata,Gervasius Remensis1055-1067,1055-1067,1532,1055.0,1067.0,False


In [23]:
cc_metadata["work_id"] = cc_metadata.index.astype(str)

In [2]:
lemmatized_metadata = pd.read_csv("../data/lemmatized_metadata.csv")

In [3]:
lemmatized_metadata

,title,author_dates,author,author_viaf,author_viaf_id,edition,editor,publisher,pub_place,pub_date,series_title,series_id,source,work_id,prelemmatized,notes
0,De ortu et obitu Patrum,565-636,Isidorus Hispalensis,http://viaf.org/viaf/803890,803890,"early modern edition, no apparatus",Jacques-Paul Migne,J. P. Migne,Parisiis,1850,"Patrologia Latina, vol. 83",IsiHis.DeOrEtO,Patrologia Latina,8209,True,NaN
1,De omnibus Sanctis,-1216,Innocentius III,http://viaf.org/viaf/16017787,16017787,"early modern edition, no apparatus",Jacques-Paul Migne,J. P. Migne,Parisiis,1855,"Patrologia Latina, vol. 217",InnIii.DeOmSa,Patrologia Latina,11823,True,NaN
2,Epistolae et privilegia,NaN,Gregorius V,http://viaf.org/viaf/78778356,78778356,"early modern edition, no apparatus",Jacques-Paul Migne,J. P. Migne,Parisiis,1853,"Patrologia Latina, vol. 137",GreV.EpEtPr15,Patrologia Latina,9537,True,NaN
3,Elegiae,c.55BC-c.19BC,Tibullus,http://viaf.org/viaf/100219266,100219266,"eclectic edition, apparatus not encoded",J. P. Postgate,Scriptorum classicorum bibliotheca Oxoniensis,NaN,1915,NaN,Tibull.Elegiae,NaN,12666,True,NaN
4,Gesta Innocentii III,NaN,Auctor incertus,NaN,NaN,"early modern edition, no apparatus",Jacques-Paul Migne,J. P. Migne,Parisiis,1855,"Patrologia Latina, vol. 214",AucInc.GeInIi,Patrologia Latina,11811,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7685,Vita S. Hildefonsi,NaN,Rodericus Cerratensis,http://viaf.org/viaf/37344826,37344826,"early modern edition, no apparatus",Jacques-Paul Migne,J. P. Migne,Parisiis,1851,"Patrologia Latina, vol. 96",RodCer.ViSHi9,Patrologia Latina,8604,True,NaN
7686,Epitaphium,NaN,Auctor incertus,NaN,NaN,"early modern edition, no apparatus",Jacques-Paul Migne,J. P. Migne,Parisiis,1849,"Patrologia Latina, vol. 72",AucInc.Epitap4,Patrologia Latina,7949,True,NaN
7687,Vita S. Gualterii,1035-1123,Marbodus Redonensis,http://viaf.org/viaf/241082057,241082057,"early modern edition, no apparatus",Jacques-Paul Migne,J. P. Migne,Parisiis,1854,"Patrologia Latina, vol. 171",MarRed.ViSGu3,Patrologia Latina,10964,True,NaN
7688,Martyrologium,-875,Ado Viennensis,http://viaf.org/viaf/73864627,73864627,"early modern edition, no apparatus",Jacques-Paul Migne,J. P. Migne,Parisiis,1852,"Patrologia Latina, vol. 123",AdoVie.Martyr6,Patrologia Latina,9187,True,['XML corrections by Jan Ctibor in 2023: isola...


In [5]:
len(lemmatized_metadata)

7690

In [6]:
lemmatized_metadata["author_viaf"].notnull().sum()

np.int64(5206)

In [16]:
lemmatized_metadata["author_viaf"] = lemmatized_metadata["author_viaf_id"]

In [17]:
viafs = lemmatized_metadata["author_viaf"].unique()
len(viafs)

1563

In [18]:
def wd_by_an_identifier(property, idstring):
    """
    Query Wikidata for an entity with the given TLG author ID (property P1266).
    Returns JSON results from the WDQS endpoint.
    """
    query = f"""
    SELECT ?item ?itemLabel WHERE {{
      ?item wdt:{property} "{idstring}" .
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    """

    url = "https://query.wikidata.org/sparql"
    headers = {"Accept": "application/sparql-results+json"}

    response = requests.get(url, params={"query": query, "format": "json"}, headers=headers)
    response.raise_for_status()

    return response.json()

In [19]:
def get_wd_authors(viaf):
    author_wd = ""
    try:
        author_wd =  wd_by_an_identifier("P214", viaf)["results"]["bindings"][0]["item"]["value"].rpartition("/")[2]
    except:
        pass
    return author_wd

In [ ]:
%%time
viaf_wd = {}
for viaf in viafs:
    viaf_wd[viaf] = get_wd_authors(viaf)

In [12]:
len(viaf_wd)

1594

In [25]:
viaf_wd

{'803890': 'Q166876',
 '16017787': 'Q128076',
 '78778356': 'Q58571',
 '100219266': 'Q109598',
 nan: '',
 '95147024': 'Q44248',
 '109786483': 'Q234593',
 '64000382': 'Q1751724',
 '268854049': 'Q577125',
 '76706732': 'Q3617330',
 '96531148': '',
 '27059664': '',
 '18013220': '',
 '100226395': '',
 '17378860': '',
 '281325874': 'Q593693',
 '1996852': '',
 '100177951': '',
 '316401002': '',
 '12311353': '',
 '160190089': '',
 '55057189': 'Q1302535',
 '100198064': '',
 '84487782': '',
 '268401625': '',
 '298713537': '',
 '72187470': '',
 '264344179': '',
 '47555113': '',
 '27422228': 'Q373675',
 '50018008': '',
 '66550784': '',
 '8182724': '',
 '34507211': '',
 '100227669': '',
 '264697612': '',
 '9966678': '',
 '100198491': 'Q551804',
 '7517943': '',
 '76751797': '',
 '5736378': '',
 '66806872': '',
 '54962011': '',
 '66569522': 'Q3460230',
 '75264960': '',
 '71412514': '',
 '95230707': '',
 '13101431': '',
 '100179090': '',
 '24572572': '',
 '29921398': '',
 '305474176': '',
 '78769600': 

In [26]:
lemmatized_metadata["author_wd"] = lemmatized_metadata["author_viaf"].apply(lambda x: viaf_wd[x])

In [27]:
lemmatized_metadata.sample(10, random_state=0)

,title,author_dates,author,author_viaf,author_viaf_id,edition,editor,publisher,pub_place,pub_date,series_title,series_id,source,work_id,prelemmatized,notes,author_wd
3736,De lapidum naturis,1035-1123,Marbodus Redonensis,241082057,241082057,"early modern edition, no apparatus",Jacques-Paul Migne,J. P. Migne,Parisiis,1854,"Patrologia Latina, vol. 171",MarRed.DeLaNa,Patrologia Latina,10950,True,NaN,
4065,De hortis,-260,Gargilius Martialis,100197892,100197892,"based on critical edition, no apparatus, onlin...",Source with summary and bibliography,DigilibLT,Vercelli,2011,digilibLT,GarMar.DeHorti,NaN,14347,True,NaN,
4166,Brevis descriptio festi gloriosissimae Virgini...,fl.1304,Pax de Ferraria,NaN,NaN,NaN,Print edition by: A. Vardanega,NaN,NaN,1929,NaN,PaxFer.BreDesFest,NaN,15375,True,"['Genere: Poesia lirica', 'Tipo: Fonti Lettera...",
528,Ibis,43BC-17,Ovidius Naso,88342447,88342447,"critical edition, apparatus not encoded",Rudolf Merkel,In Aedibus B.G. Teubneri,Lipsiae,1889,NaN,OviNas.Ibis,NaN,12707,True,NaN,
1540,Diplomata,NaN,Auctores varii,NaN,NaN,"early modern edition, no apparatus",Jacques-Paul Migne,J. P. Migne,Parisiis,1853,"Patrologia Latina, vol. 141",AucVar.Diplom11,Patrologia Latina,9860,True,NaN,
642,Commentaria in Porphyrium,480-525,Boetius,100218964,100218964,"early modern edition, no apparatus",Jacques-Paul Migne,J. P. Migne,Parisiis,1847,"Patrologia Latina, vol. 64",Boetiu.CoInPo,Patrologia Latina,7777,True,NaN,
5923,Epistola ad Philippenses,-1117,Anselmus Laudunensis et schola; Glossa ordinaria,NaN,NaN,"early modern edition, no apparatus",Jacques-Paul Migne,J. P. Migne,Parisiis,1852,"Patrologia Latina, vol. 114",WalStr.EpAdPh,Patrologia Latina,8995,True,NaN,
5151,Praecepta,NaN,Ludovicus IV Franciae; Conradus imperator,NaN,NaN,"early modern edition, no apparatus",Jacques-Paul Migne,J. P. Migne,Parisiis,1853,"Patrologia Latina, vol. 133",LuIvFrC.Praece4,Patrologia Latina,9418,True,NaN,
4120,Epistola ad Eadmerum,NaN,Nicolaus Wigorniensis,49953671,49953671,"early modern edition, no apparatus",Jacques-Paul Migne,J. P. Migne,Parisiis,1854,"Patrologia Latina, vol. 159",NicWig.EpAdEa,Patrologia Latina,10678,True,NaN,
1051,De vita Sancti Benedicti et de officio divino ...,c.1132-1202,Ioachim de Flore,78749939,78749939,NaN,Print edition by: M. Arnoni,Orizzonti meridionali,Cosenza,1994,NaN,IoaFlo.DeVitSanct,NaN,15444,True,"['Genere: Agiografia', 'Tipo: Fonti Letterarie...",Q199991


In [40]:
#lemmatized_metadata.set_index("work_id", inplace=True)

In [29]:
cc_metadata.loc[11823]

title             De omnibus Sanctis
author               Innocentius III
year                           -1216
word_count                       332
not_before                    1216.0
not_after                     1216.0
is_earlymodern                 False
work_id                        11823
Name: 11823, dtype: object

In [43]:
lemmatized_metadata.index

Index([ 8209, 11823,  9537, 12666, 11811, 14817, 11332,    52,  7160, 14403,
       ...
       10412, 11300, 10894, 11571, 10998,  8604,  7949, 10964,  9187, 10761],
      dtype='int64', name='work_id', length=7690)

In [ ]:
len(lemmatized_metadata)

In [51]:
# lemmatized_metadata.index = lemmatized_metadata["work_id"]
lemmatized_metadata["work_id"] = lemmatized_metadata.index.astype(str)

In [53]:
cc_metadata.index = cc_metadata["work_id"].astype(str)

In [60]:
cc_clean["work_id"] = cc_clean["work_id"].astype(str)
lem_clean["work_id"] = lem_clean["work_id"].astype(str)
merged = cc_clean.merge(
    lem_clean,
    on="work_id",
    how="outer",
    suffixes=("_cc", "_lem")
)


In [61]:
len(merged)

7819

In [64]:
merged[["author_cc", "author_lem", "title_cc", "title_lem"]].sample(10, random_state=0)

,author_cc,author_lem,title_cc,title_lem
1234,Bernardus Claraevallensis,Bernardus Claraevallensis,In Purificatione B. Mariae,In Purificatione B. Mariae
7518,Carolus II Calvus,Carolus II Calvus,Edictum de tributo Nordmannico,Edictum de tributo Nordmannico
5687,Leo Senonensis,Leo Senonensis,Epistola ad Childebertum,Epistola ad Childebertum
1835,Auctor incertus,Auctor incertus,Excerpta de rebus Gallicis,Excerpta de rebus Gallicis
1022,Stephanus de Balgiaco,Stephanus de Balgiaco,Tractatus de sacramento altaris,Tractatus de sacramento altaris
651,Auctor incertus (Anselmus Cantuariensis?),Auctor incertus (Anselmus Cantuariensis?),Liber de mensuratione crucis,Liber de mensuratione crucis
2726,"Chrysogonus, Federicus","Chrysogonus, Federicus",Speculum astronomicum,Speculum astronomicum
3378,Albertus Miliolus,Albertus Miliolus,Liber de temporibus et etatibus. Continuatio R...,Liber de temporibus et etatibus. Continuatio R...
3515,Bruno Signinus,Bruno Signinus,Epistolae quatuor,Epistolae quatuor
779,Conon Praenestinus,Conon Praenestinus,Epistolae,Epistolae


In [66]:
merged.to_json("/srv/data/corpus-corporum/cc_metadata.json")

In [67]:
merged.to_csv("../data/cc_metadata.csv")
